## Importy

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as TF_text
from keras.models import Model

from ImageEmbedding import ImageEmbedding
from ModelDataPreparator import ModelDataPreparator
from ImageTextEmbeddingModelFusion import ImageTextEmbeddingModelFusion

IMG_SIZE = 512
SAVED_DATA='data/saved/'

### Příprava dat

Pro každou šifru, pro kterou jsou k dispozici obrazová i týmová data vezmeme všechny týmy, které ji dokončily a vytvoříme z nich vstupní data sítě.
Data jsou převedena do numpy.arrays a rozdělena na trénovací a testovací sadu.

Datasety lze buď znovu vytvořit, nebo načíst z uložených.

## Vytvoření datové sady

In [ ]:
my_prep=ModelDataPreparator()

# načtení modelu pro klasifikaci obrázků a extrakce předposlední vrstvy, jejíž výstup chceme použít 
checkpoint_path='data/checkpoints/obtiznost/cp.ckpt'
im_model=ImageTextEmbeddingModelFusion('roberta', IMG_SIZE, True)
im_model.set_checkpoint_path(checkpoint_path)
im_model.load_weights()

embedding_model=ImageEmbedding()

layer_names=[layer.name for layer in im_model.model.layers]
intermediate_layer_model = Model(inputs=im_model.model.input,
                                 outputs=im_model.model.get_layer(layer_names[-2]).output)

source_dir='/train'
tr_st, tr_trl, tr_task, tr_lay, tr_h, tr_leg, tr_names=my_prep.create_datasets_from_folder(source_dir, intermediate_layer_model, embedding_model, resample=False)

source_dir='/val'
val_st, val_trl, val_task, val_lay, val_h, val_leg, val_names=my_prep.create_datasets_from_folder(source_dir, intermediate_layer_model, embedding_model)

### Převod dat na tensory

In [ ]:
tr_st=tf.convert_to_tensor(tr_st)
val_st=tf.convert_to_tensor(val_st)

trl_max=tr_trl.max()+1
tr_trl=tf.one_hot(tr_trl,trl_max)
val_trl=tf.one_hot(val_trl,trl_max)

task_max=tr_task.max()
tr_task=tf.one_hot(tr_task,task_max)
val_task=tf.one_hot(val_task,task_max)

tr_lay=tf.convert_to_tensor(tr_lay)
val_lay=tf.convert_to_tensor(val_lay)

tr_h_tf=tf.one_hot(tr_h, 2)
val_h_tf=tf.one_hot(val_h, 2)

## Uložení datasetů

In [ ]:
if not os.path.exists(SAVED_DATA):
    os.makedirs(SAVED_DATA) 


np.save(SAVED_DATA+"tr_stats.npy", tr_st)
np.save(SAVED_DATA+"val_stats.npy", val_st)
np.save(SAVED_DATA+"tr_trail.npy", tr_trl)
np.save(SAVED_DATA+"val_trail.npy", val_trl)
np.save(SAVED_DATA+"tr_task.npy", tr_task)
np.save(SAVED_DATA+"val_task.npy", val_task)
np.save(SAVED_DATA+"tr_layer.npy", tr_lay)
np.save(SAVED_DATA+"val_layer.npy", val_lay)
np.save(SAVED_DATA+"tr_hints.npy", tr_h)
np.save(SAVED_DATA+"val_hints.npy", val_h)
np.save(SAVED_DATA+"tr_legends.npy", tr_leg)
np.save(SAVED_DATA+"val_legends.npy", val_leg)